In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
# import pickle
import cv2
import imageio

In [2]:
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.batches import UnnormalizedBatch

In [3]:
import os
from datetime import datetime
import time
from pprint import pprint

In [4]:
plt.ioff()  # interactive mode off
# plt.ion()  # interactive mode on

# %pylab inline

np.random.seed(1728)
ia.random.seed(1728)

In [ ]:
# save
# %store batches_aug_pool

In [ ]:
# load
# %store -r batches_aug_pool

In [ ]:
# delete
# %store -d batches_aug_pool

In [5]:
DATASET_DIR = 'D:/MaskTheFace/datasets'
DATASET = os.path.join(DATASET_DIR, '')

In [6]:
# Set working directory
os.chdir(DATASET)

print('current work dir:')
print(os.getcwd())
print()
print('listdir WORK_DIR:')
pprint(os.listdir('./'))

current work dir:
D:\MaskTheFace\datasets

listdir WORK_DIR:
['Andhika_101_2021-06-04_2051 01.jpg',
 'Andhika_101_2021-06-04_2051 02.jpg',
 'Andhika_101_2021-06-04_2051 03.jpg',
 'Andhika_101_2021-06-04_2051 04.jpg',
 'Andhika_101_2021-06-04_2051 05.jpg',
 'Andhika_101_2021-06-04_2051 06.jpg',
 'Andhika_101_2021-06-04_2051 07.jpg',
 'Andhika_101_2021-06-04_2051 08.jpg',
 'Andhika_101_2021-06-04_2051 09.jpg',
 'Andhika_101_2021-06-04_2051 10.jpg',
 'Andhika_101_2021-06-04_2051 11.jpg',
 'Andhika_101_2021-06-04_2051 12.jpg',
 'Andhika_101_2021-06-04_2051 13.jpg',
 'Andhika_101_2021-06-04_2051 14.jpg',
 'Andhika_101_2021-06-04_2051 15.jpg',
 'Andhika_101_2021-06-04_2051 16.jpg',
 'Andhika_101_2021-06-04_2051 17.jpg',
 'Andhika_101_2021-06-04_2051 18.jpg',
 'Andhika_101_2021-06-04_2051 19.jpg',
 'Andhika_101_2021-06-04_2051 20.jpg',
 'Andhika_101_2021-06-04_2051 21.jpg',
 'Andhika_101_2021-06-04_2051 22.jpg',
 'Andhika_101_2021-06-04_2051 23.jpg',
 'Andhika_101_2021-06-04_2051 24.jpg',
 'A

In [ ]:
# import dill
# dill.dump_session('imgaug_env.db')

In [ ]:
# import dill
# dill.load_session('imgaug_env.db')

In [7]:
baseDatasetDirName = 'main, train set - crop resized'

def getImages(foldername):
    images = []
    folderpath = f'{DATASET:s}/_temp/{baseDatasetDirName:s}/{foldername:s}/*.JPG'
    for image_file_glob in glob.glob(folderpath):
        image = cv2.imread(image_file_glob)[:, :, ::-1]
        images.append(image)

    return np.array(images)

In [8]:
def show_images(num_col, images_aug, showfig=True):
    num_row = round(len(images_aug)/num_col)
    num = len(images_aug)
    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col, 1.65*num_row))

    # plot images
    for i in range(num_col*num_row):
        ax = axes[i//num_col, i%num_col]
        ax.axis('off')
        try:
            ax.imshow(images_aug[i])
        except IndexError:
            continue
    
    if showfig:
        plt.show(fig)
    plt.close(fig)
        
    return fig

In [9]:
def batchesPoolToFigures(batches_aug_pool, num_seed, name):
    now = datetime.now().strftime("%Y-%m-%d_%H%M") # current date and time
    print("saving batch thumbnails...")

    time_start = time.time()

    i = 1
    for batch_aug in batches_aug_pool:
        filename = f'{name:s}_{num_seed:d}_{now:s} {i:02d}.jpg'
        figure = show_images(num_col=6, images_aug=batch_aug.images_aug, showfig=False)
        figure.savefig(filename, dpi=80, bbox_inches='tight', pad_inches=0);
        plt.close(figure)
        # print("figure %d/%d saved" % (i+1, len(batches_aug_pool),))
        i += 1

    time_end = time.time()
    print("savefig done, %d figures in %.2fs" % (len(batches_aug_pool), time_end - time_start,))
    return now

In [10]:
num_seed = 101

sometimes50 = lambda aug: iaa.Sometimes(0.5, aug, seed=num_seed)
sometimes70 = lambda aug: iaa.Sometimes(0.7, aug, seed=num_seed)
sometimes85 = lambda aug: iaa.Sometimes(0.85, aug, seed=num_seed)
sometimes90 = lambda aug: iaa.Sometimes(0.9, aug, seed=num_seed)

# Augment Image
seq = iaa.Sequential([
#     iaa.Fliplr(0.5),
#     iaa.Add((-35, 35), per_channel=0.5),
#     sometimes(iaa.Affine(rotate=(-5,5)))
    sometimes70(iaa.CropAndPad(percent=(0, 0.1365), keep_size=True, pad_mode=ia.ALL, seed=num_seed)),
#     iaa.AddToHueAndSaturation((-5, 5)),
    sometimes90(iaa.WithHueAndSaturation([
        iaa.WithChannels(0, [
            iaa.Add((-5, 5), seed=num_seed)
        ], seed=num_seed),
        iaa.WithChannels(1, [
            iaa.Multiply((0.975, 1.025), seed=num_seed),
#             iaa.LinearContrast((0.9, 1.1), seed=num_seed))
        ], seed=num_seed)
    ], seed=num_seed)),
    sometimes70(iaa.OneOf([
        iaa.GaussianBlur(sigma=(0.0, 2.0), seed=num_seed),
        iaa.AverageBlur(k=(2, 5), seed=num_seed),
        iaa.BilateralBlur(d=(3, 5), sigma_color=(10, 150), sigma_space=(10, 150), seed=num_seed)
    ], seed=num_seed)),
    sometimes85(iaa.OneOf([
        iaa.GammaContrast((0.5, 2.0), seed=num_seed),
        iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6), seed=num_seed),
        iaa.LogContrast(gain=(0.6, 1.4), seed=num_seed),
        iaa.LinearContrast((0.4, 1.6), seed=num_seed)
    ], seed=num_seed))
], random_order=True, seed=num_seed)

seq_det = seq.to_deterministic()

In [11]:
# sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# seq = iaa.Sequential([
#         # apply the following augmenters to most images
#         iaa.Fliplr(0.5), # horizontally flip 50% of all images
# #         iaa.Flipud(0.2), # vertically flip 20% of all images
# #         sometimes(iaa.Affine(
# #             scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, # scale images to 80-120% of their size, individually per axis
# #             translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
# #             rotate=(-10, 10), # rotate by -45 to +45 degrees
# #             shear=(-5, 5), # shear by -16 to +16 degrees
# #             order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
# #             cval=(0, 255), # if mode is constant, use a cval between 0 and 255
# #             mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
# #         )),
#         # execute 0 to 5 of the following (less important) augmenters per image
#         # don't execute all of them, as that would often be way too strong
#         iaa.SomeOf((0, 5), [
# #             sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
#             iaa.OneOf([
#                 iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
#                 iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
#                 iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
#             ]),
#             iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), # sharpen images
#             iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
#             # search either for all edges or for directed edges,
#             # blend the result with the original image using a blobby mask
#             iaa.BlendAlphaSimplexNoise(iaa.OneOf([
#                 iaa.EdgeDetect(alpha=(0.5, 1.0)),
#                 iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
#             ])),
#             iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5), # add gaussian noise to images
#             iaa.OneOf([
#                 iaa.Dropout((0.01, 0.05), per_channel=0.2), # randomly remove up to 10% of the pixels
#                 iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
#             ]),
#             iaa.Invert(0.01, per_channel=True), # invert color channels
#             iaa.Add((-2, 2), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
# #             iaa.AddToHueAndSaturation((-1, 1)), # change hue and saturation
#             # either change the brightness of the whole image (sometimes
#             # per channel) or change the brightness of subareas
#             iaa.OneOf([
#                 iaa.Multiply((0.9, 1.1), per_channel=0.5),
#                 iaa.FrequencyNoiseAlpha(
#                     exponent=(-1, 0),
#                     first=iaa.Multiply((0.9, 1.1), per_channel=True),
#                     second=iaa.ContrastNormalization((0.9, 1.1))
#                 )
#             ]),
#             sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
#             sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
#             sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
#             ], random_order=True, seed=num_seed)
#         ], random_order=True, seed=num_seed)

# seq_det = seq.to_deterministic()

In [29]:
def main(name):
    global num_seed
    print(f"processing {name:s} ...")
    
    ###
    images = getImages(name)
    print('%d images Loaded' % len(images))
    
    ###
    NB_BATCHES = 34
    batches = [UnnormalizedBatch(images=images) for _ in range(NB_BATCHES)]
    
    print(f"Augmenting {NB_BATCHES:d} batches of {name:s} ...")
    ###
    time_start = time.time()

    with seq.pool(processes=-1, maxtasksperchild=35, seed=num_seed) as pool:
        batches_aug_pool = pool.map_batches(batches, chunksize=24)

    time_end = time.time()
    print("%d batches of augmentations done in %.2fs" % (NB_BATCHES, time_end - time_start,))
    
    ###
    timestamp = batchesPoolToFigures(batches_aug_pool, num_seed, name)
    
    print(f"{name:s} processed")
    print()
    ###
    return timestamp, batches_aug_pool

In [21]:
names = [
    'Andhika', 'Ardiyan', 'Artik', 'Ballya', 'Bina', 'Buyung',
    'Kresna', 'Mhartian', 'Raihan A', 'Syifa', 'Taufik', 'Yandi']

In [26]:
# batches_aug_pool_dict = {}

In [27]:
# for name in names[:4]:
#     batches_aug_pool_dict[name] = main(name)

processing Andhika ...
18 Images Loaded
Augmenting 35 batches of Andhika ...
35 batches of augmentations done in 50.28s
saving batch thumbnails...
savefig done, 35 figures in 45.66s
Andhika processed
processing Ardiyan ...
18 Images Loaded
Augmenting 35 batches of Ardiyan ...
35 batches of augmentations done in 51.88s
saving batch thumbnails...
savefig done, 35 figures in 44.94s
Ardiyan processed
processing Artik ...
18 Images Loaded
Augmenting 35 batches of Artik ...
35 batches of augmentations done in 51.00s
saving batch thumbnails...
savefig done, 35 figures in 48.44s
Artik processed
processing Ballya ...
18 Images Loaded
Augmenting 35 batches of Ballya ...
35 batches of augmentations done in 52.12s
saving batch thumbnails...
savefig done, 35 figures in 43.86s
Ballya processed


In [28]:
# %store batches_aug_pool_dict

Stored 'batches_aug_pool_dict' (dict)


In [30]:
# for name in names[4:8]:
#     batches_aug_pool_dict[name] = main(name)

processing Bina ...
18 images Loaded
Augmenting 35 batches of Bina ...
35 batches of augmentations done in 48.47s
saving batch thumbnails...
savefig done, 35 figures in 41.86s
Bina processed

processing Buyung ...
18 images Loaded
Augmenting 35 batches of Buyung ...
35 batches of augmentations done in 46.08s
saving batch thumbnails...
savefig done, 35 figures in 42.79s
Buyung processed

processing Kresna ...
18 images Loaded
Augmenting 35 batches of Kresna ...
35 batches of augmentations done in 45.81s
saving batch thumbnails...
savefig done, 35 figures in 42.05s
Kresna processed

processing Mhartian ...
18 images Loaded
Augmenting 35 batches of Mhartian ...
35 batches of augmentations done in 46.92s
saving batch thumbnails...
savefig done, 35 figures in 43.91s
Mhartian processed



In [31]:
# %store batches_aug_pool_dict

Stored 'batches_aug_pool_dict' (dict)


In [32]:
# for name in names[8:]:
#     batches_aug_pool_dict[name] = main(name)

processing Raihan A ...
18 images Loaded
Augmenting 35 batches of Raihan A ...
35 batches of augmentations done in 138.52s
saving batch thumbnails...
savefig done, 35 figures in 63.99s
Raihan A processed

processing Syifa ...
18 images Loaded
Augmenting 35 batches of Syifa ...
35 batches of augmentations done in 44.59s
saving batch thumbnails...
savefig done, 35 figures in 42.13s
Syifa processed

processing Taufik ...
18 images Loaded
Augmenting 35 batches of Taufik ...
35 batches of augmentations done in 45.90s
saving batch thumbnails...
savefig done, 35 figures in 41.90s
Taufik processed

processing Yandi ...
18 images Loaded
Augmenting 35 batches of Yandi ...
35 batches of augmentations done in 45.61s
saving batch thumbnails...
savefig done, 35 figures in 41.90s
Yandi processed



In [33]:
# %store batches_aug_pool_dict

Stored 'batches_aug_pool_dict' (dict)


In [57]:
# Andhika = main(names[0])
# %store Andhika
# del Andhika

# Ardiyan = main(names[1])
# %store Ardiyan
# del Ardiyan

# Artik = main(names[2])
# %store Artik
# del Artik

# Ballya = main(names[3])
# %store Ballya
# del Ballya

# Bina = main(names[4])
# %store Bina
# del Bina

# Buyung = main(names[5])
# %store Buyung
# del Buyung

# Kresna = main(names[6])
# %store Kresna
# del Kresna

# Mhartian = main(names[7])
# %store Mhartian
# del Mhartian

# Raihan_A = main(names[8])
# %store Raihan_A
# del Raihan_A

# Syifa = main(names[9])
# %store Syifa
# del Syifa


126 Images Loaded
4 batch of augmentations done in 62.33s
figure 1/4 saved
figure 2/4 saved
figure 3/4 saved
figure 4/4 saved
savefig done, 4 figures in 39.09s
Stored 'Mhartian' (list)
126 Images Loaded
4 batch of augmentations done in 55.86s
figure 1/4 saved
figure 2/4 saved
figure 3/4 saved
figure 4/4 saved
savefig done, 4 figures in 33.62s
Stored 'Raihan_A' (list)
126 Images Loaded
4 batch of augmentations done in 55.59s
figure 1/4 saved
figure 2/4 saved
figure 3/4 saved
figure 4/4 saved
savefig done, 4 figures in 38.39s
Stored 'Syifa' (list)


In [43]:
# images = getImages("Taufik")

# print('%d Images Loaded' % len(images))

126 Images Loaded


In [44]:
# NB_BATCHES = 4
# batches = [UnnormalizedBatch(images=images) for _ in range(NB_BATCHES)]

In [ ]:
# time_start = time.time()

# batches_aug = list(seq_det.augment_batches(batches, background=True))  # background=True for multicore aug

# time_end = time.time()
    
# print("Batch augmentation (background=True) done in %.2fs" % (time_end - time_start,))

In [45]:
# time_start = time.time()

# with seq.pool(processes=-1, maxtasksperchild=35, seed=num_seed) as pool:
#     batches_aug_pool = pool.map_batches(batches, chunksize=24)
    
# time_end = time.time()

# print("%d batch of augmentations done in %.2fs" % (NB_BATCHES, time_end - time_start,))

4 batch of augmentations done in 56.69s


In [46]:
# batchesPoolToFigures(batches_aug_pool=batches_aug_pool, num_seed=num_seed, name="Taufik")

figure 1/4 saved
figure 2/4 saved
figure 3/4 saved
figure 4/4 saved
savefig done, 4 figures in 34.96s


In [47]:
# save
# %store batches_aug_pool

Stored 'batches_aug_pool' (list)


In [ ]:
# _ = []

# # 1 -- show only
# time_start = time.time()

# _.append(show_images(num_col=14, images_aug=batches_aug_pool[1].images_aug, showfig=True))

# time_end = time.time()
# print("#1 overall process done in %.2fs" % (time_end - time_start,))



# i = 0
# for sliced in _:
#     print("%01d: %s, %s" % (i, type(sliced), sliced))
#     i += 1

In [25]:
SAVE_DIR = f"{DATASET_DIR:s}/Augmented Images - Unmasked"

try:
    os.mkdir(SAVE_DIR)
    print("directory successfully created")
except FileExistsError:
    print("directory already exist")

directory successfully created


In [33]:
def batchesPoolToImages(batches_aug_pool, num_seed, name):
    print(f"Saving {name:s} batches...")
    
    SAVE_PATH = f"{SAVE_DIR:s}/{name:s}"
    try:
        os.mkdir(SAVE_PATH)
    except FileExistsError:
        pass
    
    time_start = time.time()
    
    # Iterate each batch and save contained images in images_aug
    for n, batch_aug in enumerate(batches_aug_pool):
        if n+1 == 35:
            continue
        for i, image_aug in enumerate(batch_aug.images_aug):
            filename = f"{name:s}_{num_seed:d}_{n+1:d}_{i+1:003d}.jpg"
            imageio.imwrite(f"{SAVE_PATH:s}/{filename:s}", image_aug)
    
        # print(f"batch {n+1:d}/{len(batches_aug_pool):d} done")
        
    time_end = time.time()
    print("Saving %s batches done in %.2fs" % (name, time_end - time_start,))

In [12]:
# %store -r batches_aug_pool_dict

In [35]:
# for name in names:
#     batchesPoolToImages(batches_aug_pool_dict[name][1], num_seed, name)

Saving Andhika batches...
Saving Andhika batches done in 6.25s
Saving Ardiyan batches...
Saving Ardiyan batches done in 6.28s
Saving Artik batches...
Saving Artik batches done in 6.03s
Saving Ballya batches...
Saving Ballya batches done in 6.45s
Saving Bina batches...
Saving Bina batches done in 6.07s
Saving Buyung batches...
Saving Buyung batches done in 6.01s
Saving Kresna batches...
Saving Kresna batches done in 6.08s
Saving Mhartian batches...
Saving Mhartian batches done in 6.31s
Saving Raihan A batches...
Saving Raihan A batches done in 6.48s
Saving Syifa batches...
Saving Syifa batches done in 6.10s
Saving Taufik batches...
Saving Taufik batches done in 6.39s
Saving Yandi batches...
Saving Yandi batches done in 6.19s


In [93]:
# # load
# %store -r Andhika_101_20210604_0048 Ardiyan_101_20210604_0050 Artik_101_20210604_0052 Ballya_101_20210604_0054

# batchesPoolToImages(Andhika_101_20210604_0048, num_seed, "Andhika")
# batchesPoolToImages(Ardiyan_101_20210604_0050, num_seed, "Ardiyan")
# batchesPoolToImages(Artik_101_20210604_0052, num_seed, "Artik")
# batchesPoolToImages(Ballya_101_20210604_0054, num_seed, "Ballya")

# del Andhika_101_20210604_0048, Ardiyan_101_20210604_0050, Artik_101_20210604_0052, Ballya_101_20210604_0054

Saving Andhika batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Andhika batches done in 5.07s
Saving Ardiyan batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Ardiyan batches done in 5.94s
Saving Artik batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Artik batches done in 5.28s
Saving Ballya batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Ballya batches done in 5.83s


In [96]:
# # load
# %store -r Bina_101_20210604_0055 Buyung_101_20210604_0057 Kresna_101_20210604_0059 Mhartian_101_20210604_0105

# batchesPoolToImages(Bina_101_20210604_0055, num_seed, "Bine")
# batchesPoolToImages(Buyung_101_20210604_0057, num_seed, "Buyung")
# batchesPoolToImages(Kresna_101_20210604_0059, num_seed, "Kresna")
# batchesPoolToImages(Mhartian_101_20210604_0105, num_seed, "Mhartian")

# del Bina_101_20210604_0055, Buyung_101_20210604_0057, Kresna_101_20210604_0059, Mhartian_101_20210604_0105

Saving Bine batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Bine batches done in 6.14s
Saving Buyung batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Buyung batches done in 5.63s
Saving Kresna batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Kresna batches done in 5.19s
Saving Mhartian batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Mhartian batches done in 6.21s


In [97]:
# # load
# %store -r Raihan_A_101_20210604_0107 Syifa_101_20210604_0108 Taufik_101_20210604_0037 Yandi_101_20210604_0015

# batchesPoolToImages(Raihan_A_101_20210604_0107, num_seed, "RaihanA")
# batchesPoolToImages(Syifa_101_20210604_0108, num_seed, "Syifa")
# batchesPoolToImages(Taufik_101_20210604_0037, num_seed, "Taufik")
# batchesPoolToImages(Yandi_101_20210604_0015, num_seed, "Yandi")

# del Raihan_A_101_20210604_0107, Syifa_101_20210604_0108, Taufik_101_20210604_0037, Yandi_101_20210604_0015

Saving RaihanA batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving RaihanA batches done in 15.37s
Saving Syifa batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Syifa batches done in 5.61s
Saving Taufik batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Taufik batches done in 5.92s
Saving Yandi batches
batch 1/4 done
batch 2/4 done
batch 3/4 done
batch 4/4 done
Saving Yandi batches done in 5.88s
